In [1]:
_ = ! pip install web3

In [2]:
import os, sys

from datetime import datetime, timedelta
import time

import requests, json
from web3 import Web3, HTTPProvider

from eth_utils import to_bytes
from eth_keys import keys
from eth_account import Account
from eth_account.messages import SignableMessage

In [3]:
w3 = Web3(HTTPProvider("https://rpc.ankr.com/eth/1e5f9fd3e49ecec581e00815ea1eb41bc459248fd3dcf67716ff3237cc3ab68b"))
# https://1rpc.io/eth

In [4]:
source_token = "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984" # UniSwap
destination_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT

source_amount = 1000000000000000000
slippage = int(0.05 * 10000) # Convert to bps

In [5]:
eth_wallet = Account.from_key("e5ecbcfc71ad24a2131953ef9a66e4b7ed8ca9a10ffd92104e106fab1f0af4f7")

In [6]:
# 0.  One fresh quote – DO NOT fetch another one before sending
quote  = requests.get(
    "https://api.0x.org/swap/permit2/quote",
    params={
        "chainId": 1,
        "sellToken": source_token,
        "buyToken":  destination_token,
        "sellAmount": source_amount,
        "taker": eth_wallet.address,
        "slippageBps": slippage,
    },
    headers={
        "0x-version" : "v2",
        "0x-api-key" : "50cbb531-4fda-4464-acb4-5e6b5bf0e2d6"
    },
    timeout=10,
).json()

In [7]:
ERC20_ABI: list[dict] = [
    {
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
]

erc20 = w3.eth.contract(address=Web3.to_checksum_address(source_token), abi=ERC20_ABI)

sell_amount   = int(quote["sellAmount"])
permit_amount = int(quote["permit2"]["eip712"]["message"]["permitted"]["amount"])
owner_balance = int(erc20.functions.balanceOf(eth_wallet.address).call())

print ("sellAmount        :", sell_amount)
print ("permit.amount     :", permit_amount)
print ("wallet UNI balance:", owner_balance)
print ()
print ("gas fee:", (1.0e-18) * int(quote['transaction']['gas']) * int(quote['transaction']['gasPrice']), "ETH")

sellAmount        : 1000000000000000000
permit.amount     : 1000000000000000000
wallet UNI balance: 57116026654249141437

gas fee: 0.000183302618715271 ETH


In [15]:
def patch_and_repoint(data_hex: str, sig: bytes, head_index: int, ptr_value: int) -> str:
    """
    data_hex   : quote["transaction"]["data"]
    sig        : 65-byte r|s|v (v 27/28)
    head_index : 9  (from your dump)
    ptr_value  : 544 (the original relative pointer)
    """
    assert len(sig) == 65

    data = bytes.fromhex(data_hex[2:])
    selector = data[:4]
    body     = data[4:]

    # 1) Find the real 97-zero placeholder you already patched (or want to patch)
    ZERO97 = b"\x00" * 97
    off_zero97 = data.rfind(ZERO97)
    if off_zero97 == -1:
        raise ValueError("No 97-byte zero placeholder found anywhere – can’t patch safely")

    # 2) Write (65 || sig) there
    patched = (
        data[:off_zero97]
        + (65).to_bytes(32, "big")
        + sig
        + data[off_zero97 + 97:]
    )

    # 3) Re-point the head word (index = 9) to our new location
    #     ptr is relative to the start of the body (i.e. minus the 4-byte selector)
    new_ptr = off_zero97 - 4
    head_word_pos = 4 + head_index * 32
    patched = (
        patched[:head_word_pos]
        + new_ptr.to_bytes(32, "big")
        + patched[head_word_pos + 32:]
    )

    # 4) Sanity
    #    Read back the word to ensure we wrote it correctly
    read_ptr = int.from_bytes(patched[head_word_pos:head_word_pos+32], "big")
    assert read_ptr == new_ptr, "failed to rewrite head pointer"

    #    Check that at the new pointed offset we really have len=65
    abs_off = 4 + read_ptr
    assert int.from_bytes(patched[abs_off:abs_off+32], "big") == 65, "length word != 65 at new pointer"
    return "0x" + patched.hex()


# ---- use it ----
digest = bytes.fromhex(quote["permit2"]["hash"][2:])
sig    = keys.PrivateKey(eth_wallet.key).sign_msg_hash(digest).to_bytes()
if sig[-1] in (0, 1):
    sig = sig[:-1] + bytes([sig[-1] + 27])

calldata = patch_and_repoint(
    quote["transaction"]["data"],
    sig,
    head_index=9,          # from your dump
    ptr_value=544,         # from your dump
)

In [11]:
# 4. Build, sign, send
tx: dict = {
    "chainId":  1,
    "to":       Web3.to_checksum_address(quote["transaction"]["to"]),
    "data":     calldata,
    "value":    int(quote["transaction"]["value"]),
    "gas":      int(quote["transaction"]["gas"]),
    "nonce":    w3.eth.get_transaction_count(eth_wallet.address, "pending"),
}

# Prefer EIP-1559 if present
if ("maxFeePerGas" in quote["transaction"]) and ("maxPriorityFeePerGas" in quote["transaction"]):
    tx["type"] = 2
    tx["maxFeePerGas"] = int(quote["transaction"]["maxFeePerGas"])
    tx["maxPriorityFeePerGas"] = int(quote["transaction"]["maxPriorityFeePerGas"])
else:
    tx["gasPrice"] = int(quote["transaction"]["gasPrice"])

# Pass through optional fields
if "accessList" in quote["transaction"]:
    tx["accessList"] = quote["transaction"]["accessList"]

# Only for eth_call / estimateGas preflight
tx = dict(tx, **{"from": eth_wallet.address})

tx_signed = eth_wallet.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(tx_signed.raw_transaction)

NameError: name 'calldata' is not defined

In [11]:
# 5. Check result
tx_hex = w3.to_hex(tx_hash)

print("↗ broadcast :", tx_hex)
print("⏳ waiting …")
print("✅ status   :", w3.eth.wait_for_transaction_receipt(tx_hash).status)  # 1 = success

↗ broadcast : 0x2fc8bccb1bf85307dc351081cb86f524ef011501665fdad5b63129be9d0ca55e
⏳ waiting …
✅ status   : 0


In [12]:
print ("https://eth.blockscout.com/tx/" + tx_hex)

https://eth.blockscout.com/tx/0x2fc8bccb1bf85307dc351081cb86f524ef011501665fdad5b63129be9d0ca55e
